## Lấy dữ liệu từ HTML

19127216 - Đặng Hoàn Mỹ

Đối với [soundcloud.com](soundcloud.com) có nhiều cách để crawl dữ liệu. 
Sau khi tham khảo video này [How SoundCloud Influencers Grow - Data Analysis of 3,000 Publicly Scraped Artist Profiles in Python
](https://youtu.be/zL8ZF7SBWRA) thì ta có thể crawl dữ liệu bằng cách search các tên bất kì và lấy dữ liệu từ kết quả search.

1. Search bằng các chữ cái, kí tự, số và cuộn trang để lấy dữ liệu.
2. Multithread để có thể mở nhiều trang và lấy dữ liệu cùng lúc.
    Có nhiều cách để lấy dữ liệu
    * **Cách 1:** Click vào từng link để lấy nhiều thuộc tính (attributes) nhất có thể -> khá lâu vì thời gian tải trang + điều hướng trang liên tục (3 nhóm dữ liệu - playlists, users, tracks * > 1000 dòng -> > 3000 links) -> Dễ bị timeout, thời gian chạy lâu.
    * **Cách 2:** Chỉ lấy các thuộc tính xuất hiện trên trang kết quả -> Nhanh và lấy được nhiều dòng dữ liệu nhưng ít thuộc tính.
    
  => Trong file là cách thứ 2.
3. Lưu vào mảng để chứa các dòng dữ liệu cần lưu.
4. Và lưu vào các files `users.csv`, `playlists.csv`, `tracks.csv`

Nạp các thư viện cần thiết để chạy chương trình và không cần cài đặt `chromedriver.exe` để mở trang web và chạy.

Lưu ý: Vì chạy multithread nên khi chạy, máy co thể nóng và CPU, RAM tăng cao.

> ThreadPoolExecutor is an Executor subclass that uses a pool of threads to execute calls asynchronously.
> -- [concurrent.futures — Launching parallel tasks](https://docs.python.org/3/library/concurrent.futures.html#:~:text=ThreadPoolExecutor%20is%20an%20Executor%20subclass,the%20results%20of%20another%20Future%20.)

In [1]:
import sys
!pip install --upgrade selenium webdriver_manager bs4 concurrent

ERROR: Could not find a version that satisfies the requirement concurrent
ERROR: No matching distribution found for concurrent


In [2]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import selenium.webdriver.support.ui as ui
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

import time
import csv
from concurrent import futures
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin

Thiết lập WebDriver để không cần có sẵn file `chromedriver.exe` và không mở các cửa sổ trình duyệt khi chạy multithread

References:
* [How to configure ChromeDriver to initiate Chrome browser in Headless mode through Selenium?](https://stackoverflow.com/questions/46920243/how-to-configure-chromedriver-to-initiate-chrome-browser-in-headless-mode-throug)

In [3]:
#Initialize webdriver
def initialize_driver(url):
    chrome_options = webdriver.ChromeOptions()
    prefs={"profile.managed_default_content_settings.images": 2, 'disk-cache-size': 4096 }
    chrome_options.add_experimental_option("prefs", prefs) # Manage image loading and run on disk cache
    chrome_options.add_argument("--headless") # Runs Chrome in headless mode
    chrome_options.add_argument('--no-sandbox') # Bypass OS security model
    chrome_options.add_argument('--disable-dev-shm-usage') # overcome limited resource problems
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
    driver.get(url)
    return driver

Cuộn trang bằng cách chạy câu lệnh Javascript

References:
* [Làm cách nào tôi có thể cuộn một trang web bằng cách sử dụng selenium webdo trong python?
](https://qastack.vn/programming/20986631/how-can-i-scroll-a-web-page-using-selenium-webdriver-in-python)

In [4]:
#Scrolling the page
def scrolling(driver, ScrollNumber):
    for i in range(1,ScrollNumber):
        driver.execute_script("window.scrollTo(1,500000)")
        time.sleep(0.75)

Hàm lưu vào file CSV và khởi tạo các headers của các thuộc tính

In [5]:
#Write to CSV
def write_to_csv(courses_file, big_list):    
    try:
        with open(courses_file, 'a', encoding='utf-8-sig', newline='') as file:
            writer = csv.writer(file, delimiter=',')
            for list_ in big_list:
                writer.writerow(list_)
    except:
        return False

In [6]:
def initialize_csv_files(filename):
    with open(filename, 'w', encoding='utf-8-sig', newline='') as file:
        writer = csv.writer(file, delimiter=',')
        if filename == './Crawl_data/tracks.csv':
            writer.writerow(['id_song', 'song_title', 'song_link_full', 'id_artist', 'plays'])
        elif filename == './Crawl_data/users.csv':
            writer.writerow(['id_person', 'user_name', 'user_link', 'user_details', 'followers', 'tracks'])
        else:
            writer.writerow(['id_playlist', 'playlist_name', 'id_artist', 'playlist_link_full', 'songs_number'])

Khai báo các mảng và dictionary để gán ID.

ID lần lượt được tăng lên khi thêm vào mảng

In [7]:
# lưu ID cho mỗi entity
id_people = [0]
id_playlists = [0]
id_songs = [0]

# dict_ để gán ID cho từng link
urls_run_people = {}
urls_run_tracks = {}
urls_run_playlists = {}

# các link dẫn của mỗi entity
urls_playlists = []
urls_tracks = []
urls_people = []

# base link để gán khi đọc các thẻ <a>
base_link = "https://www.soundcloud.com"

## Hàm lấy dữ liệu 3 entities bằng `BeautifulSoup`

In [8]:
def crawl_playlists(driver):
    soup = BeautifulSoup(driver, "html.parser")
    for parent in soup.find_all('div', class_="sound__body"):
        content = parent.find('div', class_='sound__content')
        
        # lấy tên người tạo ra playlist và link tới tài khoản đó
        artist = content.find('a', class_='soundTitle__username')
        artist_link_full = urljoin(base_link, artist.attrs['href'])
        artist_name = artist.find('span', class_='soundTitle__usernameText').text.strip().encode('utf-8').decode('utf-8')
        # tên và link tới playlist
        playlist = content.find('a', class_='soundTitle__title')
        playlist_link_full = urljoin(base_link, playlist.attrs['href'])
        playlist_name = playlist.text.strip().encode('utf-8').decode('utf-8')
        # lấy số lượng bài hát trong playlist
        try:
            songs = content.find('a', class_='compactTrackList__moreLink')
            songs_number = ''.join(filter(str.isdigit, songs.text.strip().encode('utf-8').decode('utf-8')))
        except:
            songs = content.find_all('li', class_='compactTrackList__item')
            songs_number = len(songs)
        # chỉ lấy được tối đa 5 tracks trên playlist vì kết quả hiển thị 5 tracks 
        # trong multithread khó thao tác khi mở thêm tab mới
        for tracks in content.find_all('li', class_='compactTrackList__item'):
            try:
                artist_song = tracks.find('span', class_='compactTrackListItem__user')
                artist_song = artist_song.text.strip().encode('utf-8').decode('utf-8').replace('\n', '').replace('-', '')
            except:
                artist_song = artist_name                
            song = tracks.find('span', class_='compactTrackListItem__trackTitle')
            # nếu không lấy được bài hát thì bỏ qua
            if song is not None:
                song_title = song.text.strip().encode('utf-8').decode('utf-8')
                song_link_full = urljoin(base_link, song.attrs['data-permalink-path'])
                artist_link = urljoin(base_link, '/' + song.attrs['data-permalink-path'].split('/')[1])
                plays = tracks.find('div', class_='compactTrackListItem__additional').text.strip().encode('utf-8').decode('utf-8')
                # kiểm tra ID
                id_artist = urls_run_people.get(artist_link)
                if id_artist is None:
                    id_artist = id_people[0]
                    id_people[0] += 1
                    urls_people.append([id_artist, artist_song, artist_link, '', '', ''])
            
                id_song = urls_run_tracks.get(song_link_full)
                if id_song is None:
                    id_song = id_songs[0]
                    id_songs[0] += 1
                    urls_tracks.append([id_song, song_title, song_link_full, id_artist, plays])
                    
        id_artist = urls_run_people.get(artist_link_full)
        if id_artist is None:
            id_artist = id_people[0]
            id_people[0] += 1
            urls_people.append([id_artist, artist_name, artist_link_full, '', '', ''])
            
        id_playlist = id_playlists[0]
        id_playlists[0] += 1
        urls_run_playlists[playlist_link_full] = id_playlist
        
        urls_playlists.append([id_playlist, playlist_name, id_artist, playlist_link_full, songs_number])

In [9]:
#Crawl data fron page People to users.csv
def crawl_people(driver):
    soup = BeautifulSoup(driver, "html.parser")
    for parent in soup.find_all('div', class_="userItem"):
        user_link = parent.find('a', class_='userItem__coverArt')
        user_link_full = urljoin(base_link, user_link.attrs['href'])
        # tên và additional info của user
        user_name = parent.find('h2', class_='userItem__title').text.strip().encode('utf-8').decode('utf-8').replace('\n', '').replace('Verified', '')
        user_details = parent.find('h3', class_='userItem__details').text.strip().encode('utf-8').decode('utf-8')
        user_details = user_details.replace('\n', ' ')
        user_details = user_details.replace('          ', ', ')
        
        set_str = parent.find_all('li', class_='sc-ministats-item')
        try:
            followers, tracks = [''.join(filter(str.isdigit, a.attrs['title'])) for a in set_str]
        except:
            followers, tracks = 0, 0
            
        id_person = id_people[0]
        id_people[0] += 1
        urls_run_people[user_link_full] = id_person
        urls_people.append([id_person, user_name, user_link_full, user_details, followers, tracks])

Khởi tạo chạy multithread từ thư viện concurrent, lấy các users, nếu lấy nhiều hơn thì tăng số lần cuộn trang, để tránh trường hợp bị trùng lặp ID khi gán thì `max_workers` sẽ là 15 - số cửa sổ tối đa được mở và chạy cùng lúc

## Lấy dữ liệu các users

Bằng cách search các kí tự, ta thu thập kết quả từ trên đó

In [10]:
start = time.time()
search_base_link = 'https://soundcloud.com/search/people?q='
letters = 'abcdefghijklmnopqrstuvwxyz!@#$%&*)(_=+-/0123456789'
search_links = [search_base_link + letter for letter in letters]

def get_users(url):
    browser = initialize_driver(url)
    
    element = WebDriverWait(browser, 20).until(
        EC.presence_of_element_located((By.ID, 'onetrust-accept-btn-handler'))
    )
    element.click()
    scrolling(browser, 7)

    html_source = browser.page_source
    browser.quit()
    
    crawl_people(html_source)
    
with futures.ThreadPoolExecutor(max_workers=15) as executor:
    executor.map(get_users, search_links)



====== WebDriver manager ======


====== WebDriver manager ======














====== WebDriver manager ======




====== WebDriver manager ======


====== WebDriver manager ======






====== WebDriver manager ======
====== WebDriver manager ======
====== WebDriver manager ======
====== WebDriver manager ======
====== WebDriver manager ======
Current google-chrome version is 95.0.4638
====== WebDriver manager ======
Current google-chrome version is 95.0.4638
====== WebDriver manager ======
====== WebDriver manager ======
====== WebDriver manager ======
====== WebDriver manager ======
Get LATEST chromedriver version for 95.0.4638 google-chrome
Get LATEST chromedriver version for 95.0.4638 google-chrome
Current google-chrome version is 95.0.4638
Get LATEST chromedriver version for 95.0.4638 google-chrome
Current google-chrome version is 95.0.4638
Current google-chrome version is 95.0.4638
Get LATEST chromedriver version for 95.0.4638 google-chrome
Get LATEST chromedriver version for



Current google-chrome version is 95.0.4638
====== WebDriver manager ======
Get LATEST chromedriver version for 95.0.4638 google-chrome
Current google-chrome version is 95.0.4638
Get LATEST chromedriver version for 95.0.4638 google-chrome
Driver [C:\Users\ryedh\.wdm\drivers\chromedriver\win32\95.0.4638.69\chromedriver.exe] found in cache
Driver [C:\Users\ryedh\.wdm\drivers\chromedriver\win32\95.0.4638.69\chromedriver.exe] found in cache
Driver [C:\Users\ryedh\.wdm\drivers\chromedriver\win32\95.0.4638.69\chromedriver.exe] found in cache


====== WebDriver manager ======


====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST chromedriver version for 95.0.4638 google-chrome
Current google-chrome version is 95.0.4638
Get LATEST chromedriver version for 95.0.4638 google-chrome
Driver [C:\Users\ryedh\.wdm\drivers\chromedriver\win32\95.0.4638.69\chromedriver.exe] found in cache


====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get L

Thời gian nhanh hơn nhiều so với chạy đơn luồng bình thường (tính bằng giây)

In [11]:
stop = time.time()
print(stop - start)

74.61343884468079


In [12]:
urls_people

[[0,
  'Legacy Recordings',
  'https://www.soundcloud.com/legacyrecordings',
  'Legacy Recordings, New York City / United States',
  '3455211',
  '1'],
 [1,
  'Los Campesinos!',
  'https://www.soundcloud.com/los-campesinos',
  'Los Campesinos!, Cardiff / United Kingdom',
  '4748064',
  '167'],
 [2,
  'Lang Lang',
  'https://www.soundcloud.com/lang-lang',
  'Lang Lang, China',
  '1961110',
  '256'],
 [3,
  'Los Bunkers',
  'https://www.soundcloud.com/los-bunkers',
  'Concepción / Chile',
  '796826',
  '172'],
 [4,
  'Lions Tour 2013',
  'https://www.soundcloud.com/lions-tour-2013',
  '',
  '191340',
  '49'],
 [5,
  'LeonaLewis',
  'https://www.soundcloud.com/leonalewis',
  'Leona Lewis, London / United Kingdom',
  '211819',
  '28'],
 [6, 'LBCradio', 'https://www.soundcloud.com/lbcradio', '', '298809', '13'],
 [7,
  'Letter from America 93-96',
  'https://www.soundcloud.com/letter-clinton',
  'Broadcasting House, London / United Kingdom',
  '413912',
  '13'],
 [8,
  'InnerCat Music Group

In [13]:
print(len(urls_people), len(urls_people[0]))
type(urls_people[0])

1980 6


list

## Lấy dữ liệu playlists và users

Khởi tạo chạy multithread từ thư viện concurrent, lấy các playlists và các tracks, nếu lấy nhiều hơn thì tăng số lần cuộn trang, để tránh trường hợp bị trùng lặp ID khi gán thì `max_workers` sẽ là 15 - số cửa sổ tối đa được mở và chạy cùng lúc

In [14]:
start = time.time()
search_base_link = 'https://soundcloud.com/search/sets?q='
letters = 'abcdefghijklmnopqrstuvwxyz!@#$%&*)(_=+-/0123456789'
# letters = 'abc'
search_links = [search_base_link + letter for letter in letters]

def get_playlists(url):
    browser = initialize_driver(url)
    
    element = WebDriverWait(browser, 20).until(
        EC.presence_of_element_located((By.ID, 'onetrust-accept-btn-handler'))
    )
    element.click()
    scrolling(browser, 7)
    html_source = browser.page_source
    browser.quit()
    
    crawl_playlists(html_source)
    
with futures.ThreadPoolExecutor(max_workers=15) as executor:
    executor.map(get_playlists, search_links)



====== WebDriver manager ======








====== WebDriver manager ======




====== WebDriver manager ======




====== WebDriver manager ======


====== WebDriver manager ======






====== WebDriver manager ======


====== WebDriver manager ======


====== WebDriver manager ======
====== WebDriver manager ======
====== WebDriver manager ======
====== WebDriver manager ======
====== WebDriver manager ======
====== WebDriver manager ======
====== WebDriver manager ======
Current google-chrome version is 95.0.4638
====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Current google-chrome version is 95.0.4638
Get LATEST chromedriver version for 95.0.4638 google-chrome
Current google-chrome version is 95.0.4638
Get LATEST chromedriver version for 95.0.4638 google-chrome
Get LATEST chromedriver version for 95.0.4638 google-chrome
Get LATEST chromedriver version for 95.0.4638 google-chrome
Current google-chrome version is 95.0.4638
Current google-chrome version is 95

Driver [C:\Users\ryedh\.wdm\drivers\chromedriver\win32\95.0.4638.69\chromedriver.exe] found in cache
Driver [C:\Users\ryedh\.wdm\drivers\chromedriver\win32\95.0.4638.69\chromedriver.exe] found in cache


====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST chromedriver version for 95.0.4638 google-chrome


====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST chromedriver version for 95.0.4638 google-chrome
Driver [C:\Users\ryedh\.wdm\drivers\chromedriver\win32\95.0.4638.69\chromedriver.exe] found in cache
Driver [C:\Users\ryedh\.wdm\drivers\chromedriver\win32\95.0.4638.69\chromedriver.exe] found in cache


====== WebDriver manager ======


====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST chromedriver version for 95.0.4638 google-chrome
Current google-chrome version is 95.0.4638
Get LATEST chromedriver version for 95.0.4638 google-chrome
Driver [C:\Users\ryedh\.wdm\drivers\chromedriver

In [15]:
stop = time.time()
print(stop - start)

85.44782519340515


In [16]:
urls_tracks

[[0,
  'French Montana - Pop That feat. Rick Ross, Drake & Lil Wayne (Explicit)',
  'https://www.soundcloud.com/badboyentertainment/french-montana-pop-that-feat?in=badboyentertainment/sets/frenchmontanamusic-com-player',
  1982,
  '3.12M'],
 [1,
  'أهواك أهواك فيروز Fairouz ahwak',
  'https://www.soundcloud.com/tarazalrayhan/fairouz-ahwak?in=marrram/sets/fairuz-3',
  1984,
  '8.89M'],
 [2,
  'MaakAlby - Amr Diab / insta @salamkarim_sk',
  'https://www.soundcloud.com/salamkarim_sk/maakalby?in=sabrina-sabrina-6551471/sets/fatma',
  1986,
  '2.76M'],
 [3,
  'Amr Diab 7abibty + La La',
  'https://www.soundcloud.com/ramy-pop-top/amr-diab-7abibty?in=sabrina-sabrina-6551471/sets/fatma',
  1987,
  '930K'],
 [4,
  'Amr Diab - Keda 3eny 3enak عمرو دياب - كده عيني عينك',
  'https://www.soundcloud.com/ma227/amr-diab-keda-3eny-3enak?in=sabrina-sabrina-6551471/sets/fatma',
  1988,
  '10.7M'],
 [5,
  'Ahla Kalam - Haitham Nabil احلى كلام  - هيثم نبيل.mp3',
  'https://www.soundcloud.com/m-stafa-s-bhy/

In [17]:
print(len(urls_tracks), len(urls_tracks[0]))

6655 5


In [18]:
urls_playlists

[[0,
  'fav',
  1980,
  'https://www.soundcloud.com/rania-atef-986053249/sets/fav',
  0],
 [1,
  'favorite',
  1981,
  'https://www.soundcloud.com/abdul-salam-888480241/sets/favorite',
  0],
 [2,
  'FrenchMontanaMusic.com Player',
  1983,
  'https://www.soundcloud.com/badboyentertainment/sets/frenchmontanamusic-com-player',
  1],
 [3, 'fairuz <3', 1985, 'https://www.soundcloud.com/marrram/sets/fairuz-3', 1],
 [4,
  'fatma',
  1991,
  'https://www.soundcloud.com/sabrina-sabrina-6551471/sets/fatma',
  '9'],
 [5,
  'First on SoundCloud Clubhouse Sessions',
  1997,
  'https://www.soundcloud.com/soundcloud/sets/first-on-soundcloud-clubhouse-sessions',
  '8'],
 [6,
  'FlatbushZOMBiES - BetterOffDEAD',
  2003,
  'https://www.soundcloud.com/flatbushzombies/sets/betteroffdead',
  '19'],
 [7,
  'Feduk',
  2009,
  'https://www.soundcloud.com/user-477709787/sets/feduk',
  '136'],
 [8,
  'favoritos',
  2015,
  'https://www.soundcloud.com/guillermo-ramirez-20/sets/favoritos',
  '24'],
 [9,
  'Favs',

In [19]:
print(len(urls_playlists), len(urls_playlists[0]))

1450 5


## Ghi vào file và đọc lại file để kiểm tra

In [20]:
initialize_csv_files('./Crawl_data/tracks.csv')
initialize_csv_files('./Crawl_data/playlists.csv')
initialize_csv_files('./Crawl_data/users.csv')

In [21]:
write_to_csv('./Crawl_data/users.csv', urls_people)
write_to_csv('./Crawl_data/tracks.csv', urls_tracks)
write_to_csv('./Crawl_data/playlists.csv', urls_playlists)

## Đọc file lên kiểm tra

In [22]:
import pandas as pd
df_users = pd.read_csv('./Crawl_data/users.csv')
df_playlists = pd.read_csv('./Crawl_data/playlists.csv')
df_tracks = pd.read_csv('./Crawl_data/tracks.csv')

`id_person` ID được gán lần lượt theo thứ tự lưu vào mảng - vì không thể lấy được ID khi parse HTML

`user_name` tên các người dùng

`user_link` đường dẫn đến trang cá nhân của người dùng

`user_details` thông tin đề thêm từ người dùng, thể hiện vị trí và thông tin thêm (additional information $\neq$ description)

`followers` số lượng người theo dõi

`tracks` số lượng tracks mà người này đã đăng tải

In [23]:
df_users

,id_person,user_name,user_link,user_details,followers,tracks
0,0,Legacy Recordings,https://www.soundcloud.com/legacyrecordings,"Legacy Recordings, New York City / United States",3455211.0,1.0
1,1,Los Campesinos!,https://www.soundcloud.com/los-campesinos,"Los Campesinos!, Cardiff / United Kingdom",4748064.0,167.0
2,2,Lang Lang,https://www.soundcloud.com/lang-lang,"Lang Lang, China",1961110.0,256.0
3,3,Los Bunkers,https://www.soundcloud.com/los-bunkers,Concepción / Chile,796826.0,172.0
4,4,Lions Tour 2013,https://www.soundcloud.com/lions-tour-2013,NaN,191340.0,49.0
...,...,...,...,...,...,...
9874,9874,BabyE,https://www.soundcloud.com/babye,NaN,NaN,NaN
9875,9875,KingFaro,https://www.soundcloud.com/kingfaro,NaN,NaN,NaN
9876,9876,Deep Ellum Music Group,https://www.soundcloud.com/deepellummusic,NaN,NaN,NaN
9877,9877,Roy Woods,https://www.soundcloud.com/roywoodsofficial,NaN,NaN,NaN


`id_playlist` ID được gán lần lượt theo thứ tự lưu vào mảng - vì không thể lấy được ID khi parse HTML

`playlist_name` tên của playlist

`id_artist` ID của người tạo ra playlist này

`playlist_link_full` đường dẫn tới trang của playlist này

`songs_number` số lượng bài hát trong playlist

Vì chỉ crawl ở trang kết quả khi search nên không thể lấy đầy đủ các bài hát, nếu được thì sẽ là một mảng chứa ID của các bài hát trong playlist.

In [24]:
df_playlists

,id_playlist,playlist_name,id_artist,playlist_link_full,songs_number
0,0,fav,1980,https://www.soundcloud.com/rania-atef-98605324...,0
1,1,favorite,1981,https://www.soundcloud.com/abdul-salam-8884802...,0
2,2,FrenchMontanaMusic.com Player,1983,https://www.soundcloud.com/badboyentertainment...,1
3,3,fairuz <3,1985,https://www.soundcloud.com/marrram/sets/fairuz-3,1
4,4,fatma,1991,https://www.soundcloud.com/sabrina-sabrina-655...,9
...,...,...,...,...,...
1445,1445,50 carrot_part3,9854,https://www.soundcloud.com/emmanuel-piolet-562...,407
1446,1446,50 carrot_part3,9860,https://www.soundcloud.com/emmanuel-piolet-562...,407
1447,1447,5555,9866,https://www.soundcloud.com/marcos-lamar-993779...,485
1448,1448,5 o clock somewhere,9872,https://www.soundcloud.com/user-188183230/sets...,424


`id_song` ID được gán lần lượt theo thứ tự lưu vào mảng - vì không thể lấy được ID khi parse HTML

`song_title` tên của track

`song_link_full` đường dẫn tới trang của track này

`id_artist` ID của người đăng tải track

`plays` số lượt chơi của track này

In [25]:
df_tracks

,id_song,song_title,song_link_full,id_artist,plays
0,0,"French Montana - Pop That feat. Rick Ross, Dra...",https://www.soundcloud.com/badboyentertainment...,1982,3.12M
1,1,أهواك أهواك فيروز Fairouz ahwak,https://www.soundcloud.com/tarazalrayhan/fairo...,1984,8.89M
2,2,MaakAlby - Amr Diab / insta @salamkarim_sk,https://www.soundcloud.com/salamkarim_sk/maaka...,1986,2.76M
3,3,Amr Diab 7abibty + La La,https://www.soundcloud.com/ramy-pop-top/amr-di...,1987,930K
4,4,Amr Diab - Keda 3eny 3enak عمرو دياب - كده عين...,https://www.soundcloud.com/ma227/amr-diab-keda...,1988,10.7M
...,...,...,...,...,...
6650,6650,Chill Bill (feat. J. Davi$ & Spooks),https://www.soundcloud.com/robstone1207/chill-...,9873,Not available in Viet Nam
6651,6651,BABY E. - FINESSIN REMIX FT. KEVIN GATES & LIL...,https://www.soundcloud.com/babye/finessinremix...,9874,6.09M
6652,6652,autoLove.,https://www.soundcloud.com/kingfaro/autolove?i...,9875,"9,851"
6653,6653,Nature Nate - Dallas And Beyond (prod Campion ...,https://www.soundcloud.com/deepellummusic/natu...,9876,1.29M
